多输入多输出通道 channels
- 通道数是大家会仔细取设的超参数

多个输入通道
- 每个通道都有一个卷积核，结果是所有卷积结果的和
- 这样输出是个单通道

多个输出通道
- 无论有多少输入通道，到目前为止我们只用到单输出通道
- 我们可以有多个三维卷积核，每个核生成一个输出通道
- 输出里面的第i个通道，就是完整的输入和我对应的第i个核，也就是3D的核矩阵做多输入的卷积，整体上是4D
- 输入和输出通道没有太多的相关性

我们为什么要干这个事情 - 多个输入和输出通道
- 每个输出通道可以识别特定模式
- 输入通道核识别并组合输入中的模式 - 加权后一加

1x1卷积层
- 高和宽都是1的卷积层是一个受欢迎的选择。它不识别空间模式，只是融合通道。
- 等价于把输入拉成一个向量，输入通道数拉成一个feature数

二维卷积层
- 计算复杂度（浮点计算数FLOP）O(ci co kh kw mh mw)
- ci=co=100 通道数等于100
- kh=kw=5 核宽高
- mh=mw=64 输出宽高
- 1GFLOPs = 10亿个浮点运算
- 10层，1M样本，10PFlops（CPU：0.15TF/s=18h， GPU：12TF/s=14min）
- 卷积的模型挺小的，所以存储挺小的，但是计算量一点都不小

总结
- 输出通道数是卷积层的超参数，输入不是
- 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果
- 每个输出通道有独立的三维卷积核，所以最好卷积核是一个四维的张量


答疑
- 假设输入输出高宽不变的情况下，通常不会动通道数的值。
- 假设高宽都减半的情况下，通常我会把输出的通道数加一倍。我把空间信息压缩了，提取出来的信息在更多的通道上存下来。
- padding 0越多，计算性能只加了一点点，模型性能不会影响，0不会对输出产生影响。0和卷积一乘就得到0，虽然会有偏差，等价于一个常数。
- 不同通道的卷积核大小通常是一样的，这是因为计算上的好处。如果不一样写成两个卷积操作，现在一个就行。
- 偏移的作用越来越低。当数据的均值不为0的时候，偏移就等于均值的负数。因为我们会做大量均一化的操作，所以偏移实际上没太大影响。
- 核的参数是学出来的，不是选出来的。
- 这一节介绍的是二维卷积，如果是3d的用3维卷积，例如深度图。RGB就是通道数了，还是2维。
- 1x1卷积核每个输出的元素只看了对应的那个输入的像素，没有看任何输入的像素周围边上这一圈东西，只看了自己，根本没有看这个像素跟边上像素的关系是什么样子。这就是1x1卷积层不识别空间信息的由来。
- 卷积能有位置信息，输出第ij就是对应输入那个位置周围的信息。
- 每个通道学习不一样的东西，学习特定的模式，通道之间不共享参数。
- 所谓的feature map就是卷积的输出。


MobileNet
- 设计思路：depth width convolution
- 先对每个输入通道做3x3的卷积，不想之前把他加起来再做输出通道，再对每一个做空间融合
- 好处是计算量比较小，适合移动端的神经网络，计算复杂度非常低，效果现在可以到很不错的



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

In [ ]:
# 多输入多输出通道
# 实现一下多输入通道互相关运算

import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历X和K的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [ ]:
# 验证互相关运算的输出
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [ ]:
# 计算多个通道的输出的互相关函数
def corr2d_multi_in_out(X, K):
    # 迭代K的第0个维度，每次都对输入X执行互相关运算
    # 最好将所有结果都叠加在一起
    # K是4D的，k是3D的tensor
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [ ]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [ ]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [ ]:
# 1x1卷积等价于一个全连接
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    # 拉成一条向量
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    # 之前是反过来写的 这里我们转置了
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [ ]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [ ]:
# pytorch怎么调用呢？
# 将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

In [ ]:
# 一个稍微复杂的例子
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3,4))
comp_conv2d(conv2d, X).shape